In [1]:
!pip install pycaret

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from pycaret.classification import *
import pandas as pd 
train=pd.read_csv('train.csv',index_col=0)
test=pd.read_csv('test_x.csv',index_col=0)

In [3]:
def tp_score(train):
    train['diligence']=(train['tp03']+ 8-train['tp08'])/2
    train['Affinity']=(train['tp07']+ 8-train['tp02'])/2
    train['emotional stability']=(train['tp09']+ 8-train['tp04'])/2
    train['openness']=(train['tp05']+ 8-train['tp10'])/2
    train['extroversion']=(train['tp01']+ 8-train['tp06'])/2
    train.drop([ 'tp01', 'tp02', 'tp03', 'tp04', 'tp05','tp06', 'tp07', 'tp08', 'tp09', 'tp10'],axis=1,inplace=True)
    return train
train=tp_score(train)
test=tp_score(test)

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
train1=pd.concat([train[train.columns.difference(['gender', 'age_group', 'race', 'religion','voted'])] ,pd.get_dummies(train[['gender', 'age_group', 'race', 'religion']])],axis=1)
X=train1 #(45532, 100) w/o voted
scaler.fit(X)
X=scaler.transform(X)
train=pd.concat([pd.DataFrame(X),train['voted']],axis=1) #타겟도 같이 벡터화 기기

test=pd.concat([test[test.columns.difference(['gender', 'age_group', 'race', 'religion'])] ,pd.get_dummies(test[['gender', 'age_group', 'race', 'religion']])],axis=1)
test=scaler.transform(test)

In [4]:
import numpy as np
clf = setup(data = train, target = 'voted')

Setup Succesfully Completed!


,Description,Value
0,session_id,6027
1,Target Type,Binary
2,Label Encoded,"1: 0, 2: 1"
3,Original Data,"(45532, 96)"
4,Missing Values,False
5,Numeric Features,51
6,Categorical Features,44
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [5]:
lgbm=create_model('lightgbm',learning_rate=0.1) 
gbc=create_model('gbc',learning_rate=0.1) 
catboost=create_model('catboost',learning_rate=0.1) 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6797,0.7479,0.6660,0.7257,0.6946,0.3591,0.3606
1,0.6814,0.7548,0.6654,0.7285,0.6955,0.3629,0.3645
2,0.6772,0.7447,0.6546,0.7279,0.6893,0.3555,0.3576
3,0.6807,0.7497,0.6648,0.7278,0.6949,0.3615,0.3631
4,0.6858,0.7538,0.6735,0.7308,0.7010,0.3712,0.3726
5,0.6998,0.7699,0.6754,0.7505,0.7110,0.4005,0.4029
6,0.6820,0.7517,0.6588,0.7326,0.6938,0.3650,0.3672
7,0.6806,0.7492,0.6665,0.7267,0.6953,0.3611,0.3626
8,0.6841,0.7548,0.6658,0.7321,0.6974,0.3685,0.3703
9,0.6740,0.7449,0.6628,0.7192,0.6899,0.3475,0.3488


In [6]:
blended = blend_models([lgbm,gbc,catboost], fold = 10, method = 'soft') 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6975,0.7611,0.6539,0.7594,0.7027,0.3982,0.4028
1,0.6988,0.7681,0.6577,0.7594,0.7049,0.4007,0.4050
2,0.6926,0.7518,0.6469,0.7558,0.6971,0.3888,0.3937
3,0.6982,0.7640,0.6514,0.7621,0.7024,0.4000,0.4051
4,0.7026,0.7651,0.6620,0.7627,0.7088,0.4080,0.4123
5,0.7047,0.7779,0.6543,0.7708,0.7078,0.4133,0.4190
6,0.6953,0.7610,0.6448,0.7613,0.6982,0.3947,0.4003
7,0.6879,0.7611,0.6429,0.7506,0.6925,0.3796,0.3843
8,0.6980,0.7704,0.6480,0.7639,0.7012,0.4002,0.4057
9,0.6879,0.7560,0.6444,0.7500,0.6932,0.3794,0.3839


In [7]:
predict_model(blended) 

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.6897,0.762,0.6423,0.7538,0.6936,0.3833,0.3884


,0,1,2,3,4,5,6,7,8,9,...,91_1.0,92_0.0,92_1.0,93_0.0,93_1.0,94_0.0,94_1.0,voted,Label,Score
0,0.357143,0.25,0.000253,0.75,0.000226,0.75,0.001143,0.75,0.001563,0.00,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1,2,0.7826
1,0.571429,0.00,0.000153,1.00,0.000181,1.00,0.000420,0.00,0.000375,1.00,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1,1,0.4774
2,0.285714,0.00,0.000201,0.50,0.000122,1.00,0.000517,0.00,0.000431,0.25,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2,1,0.4659
3,0.285714,1.00,0.000150,0.50,0.000399,0.25,0.001199,0.00,0.000551,1.00,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2,2,0.8721
4,0.500000,0.25,0.000136,1.00,0.000101,0.00,0.000709,0.00,0.000379,0.00,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2,2,0.9802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0.428571,0.25,0.000351,0.00,0.000262,1.00,0.000862,0.00,0.000554,0.00,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1,1,0.3154
12290,0.571429,0.00,0.000181,0.50,0.000190,1.00,0.000998,0.00,0.000962,0.00,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1,1,0.2725
12291,0.500000,0.25,0.000633,1.00,0.000366,0.00,0.000754,0.25,0.000334,0.00,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2,1,0.2538
12292,0.357143,0.25,0.001170,0.00,0.000612,0.75,0.004555,0.00,0.002143,0.00,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2,2,0.5316


In [8]:
final_model=finalize_model(blended)

In [9]:
test=pd.DataFrame(test)

In [10]:
#제출용 
predict=predict_model(final_model,data=test) 

In [11]:
submission=pd.DataFrame(data={'voted':predict['Score']})
submission.reset_index(inplace=True)
submission.to_csv('submission/submission12.csv',index=False)